In [1]:
import pandas as pd
import requests
import json
import uuid
import os
import time

### 调用api

In [3]:
API_KEY = "your api"

In [6]:
# 替换文件路径
FILE_PATH = "/Users/hemengya/Desktop/数字贸易规则新版/第五版/embedding/processed_data.csv"

# 指定输入和输出的列名
INPUT_COLUMN = "待嵌入"
OUTPUT_COLUMN = "判断"

# 指定唯一标识列名，如果不存在会自动创建
KEY_COLUMN = "key"

# 缓存文件路径
CACHE_FILE = "/Users/hemengya/Desktop/数字贸易规则新版/第五版/qwen/cache.json"

# 设置 API 的 URL
API_URL = "https://api.siliconflow.cn/v1/chat/completions"

# 每次 API 调用之间的延迟（秒），用于避免速率限制
REQUEST_DELAY = 1  # 初始延迟，可以根据需要调整

# 批处理大小
BATCH_SIZE = 10

# --- 辅助函数：缓存操作 ---
def load_cache():
    """从 JSON 文件加载缓存。"""
    if os.path.exists(CACHE_FILE):
        with open(CACHE_FILE, 'r', encoding='utf-8') as f:
            return json.load(f)
    return {}

def save_cache(cache):
    """将缓存保存到 JSON 文件。"""
    with open(CACHE_FILE, 'w', encoding='utf-8') as f:
        json.dump(cache, f, ensure_ascii=False, indent=4)

# --- API 调用函数 ---
def call_api_batch(texts, api_key):
    """
    向 API 发送批量请求并返回结果。
    """
    system_prompt = """你是一个国际贸易协议、国际贸易法研究的专家。现在请阅读我给你的文段，并严格按照以下标准，摘出与“数字贸易”相关的条款、规定的句子： 
 1. **内容聚焦：** 只截取明确与“数字贸易”、“数字产品”等相关的句子。如果只是泛泛的贸易条款，并非清晰指向数字贸易，请勿输出。
2. **参考标准：** 在判断句子是否与数字贸易相关时，请参考以下数字贸易规则在国际贸易领域中通常体现的条款类型：
 * 市场准入，涉及数字产品的处理和责任;
* 促进数字化贸易和贸易便利化——此类规定支持和便利通过电子手段完成的贸易程序，例如促进使用自动化海关系统; 
 * 用户保护与网络安全，包括个人信息保护、网络消费者保护、网络安全保护等;
 * 数字知识产权相关问题; 
 * 包容性数字贸易，例如数字贸易条款，包括对中小微企业的特殊考虑; 
 * 其他新出现的问题，如政府数据公开和数据创新; 
 * 争议解决，规定是否有适用于数字贸易条款、数字贸易规则产生的争议的争议解决机制。
 3. **原文输出：** 输出的句子必须是原文，也就是英文原文，不做任何修改、推断或添加您的思考过程。 
4. **格式要求：** 如果有多个相关句子，请将它们前后拼接为一个段落输出。
注意，输出只要原文句子，不要你的任何补充、解释与说明。如果该列没有相关的句子，请输出空格。"""

    results = []
    current_delay = REQUEST_DELAY
    
    for text in texts:
        payload = {
            "model": "deepseek-ai/DeepSeek-R1",
            "messages": [
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": text
                }
            ]
        }
        headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }

        while True:
            try:
                response = requests.post(API_URL, json=payload, headers=headers)
                response.raise_for_status()
                result = response.json().get('choices', [])[0]['message']['content']
                results.append(result.strip())
                current_delay = REQUEST_DELAY 
                break
            except requests.exceptions.HTTPError as e:
                if e.response.status_code == 429:
                    print(f"检测到速率限制 (429)。正在退避... 等待 {current_delay} 秒。")
                    time.sleep(current_delay)
                    current_delay *= 2  
                    if current_delay > 60:  
                        current_delay = 60
                else:
                    print(f"API 请求失败: {e}")
                    results.append(f"请求失败: {e}")
                    current_delay = REQUEST_DELAY 
                    break
            except requests.exceptions.RequestException as e:
                print(f"API 请求失败: {e}")
                results.append(f"请求失败: {e}")
                current_delay = REQUEST_DELAY
                break
            
        time.sleep(REQUEST_DELAY) # 每次调用后都等待
            
    return results

开始处理 CSV 文件...
正在处理一个包含 10 条记录的批次...
正在处理一个包含 10 条记录的批次...
正在处理一个包含 10 条记录的批次...
正在处理一个包含 10 条记录的批次...
正在处理一个包含 10 条记录的批次...
正在处理一个包含 10 条记录的批次...
检测到速率限制 (429)。正在退避... 等待 1 秒。
检测到速率限制 (429)。正在退避... 等待 2 秒。
检测到速率限制 (429)。正在退避... 等待 4 秒。
检测到速率限制 (429)。正在退避... 等待 8 秒。
检测到速率限制 (429)。正在退避... 等待 1 秒。
检测到速率限制 (429)。正在退避... 等待 2 秒。
检测到速率限制 (429)。正在退避... 等待 4 秒。
检测到速率限制 (429)。正在退避... 等待 8 秒。
检测到速率限制 (429)。正在退避... 等待 1 秒。
检测到速率限制 (429)。正在退避... 等待 2 秒。
检测到速率限制 (429)。正在退避... 等待 4 秒。
检测到速率限制 (429)。正在退避... 等待 1 秒。
检测到速率限制 (429)。正在退避... 等待 2 秒。
检测到速率限制 (429)。正在退避... 等待 4 秒。
检测到速率限制 (429)。正在退避... 等待 1 秒。
检测到速率限制 (429)。正在退避... 等待 2 秒。
检测到速率限制 (429)。正在退避... 等待 4 秒。
检测到速率限制 (429)。正在退避... 等待 1 秒。
检测到速率限制 (429)。正在退避... 等待 2 秒。
检测到速率限制 (429)。正在退避... 等待 4 秒。
检测到速率限制 (429)。正在退避... 等待 1 秒。
检测到速率限制 (429)。正在退避... 等待 1 秒。
检测到速率限制 (429)。正在退避... 等待 1 秒。
检测到速率限制 (429)。正在退避... 等待 2 秒。
检测到速率限制 (429)。正在退避... 等待 4 秒。
检测到速率限制 (429)。正在退避... 等待 8 秒。
检测到速率限制 (429)。正在退避... 等待 16 秒。
检测到速率限制 (429)。正在退避... 等待 32 秒。
正在处理一个包含 10

### 运行

In [ ]:

# --- 主程序 ---
def process_csv_with_cache(file_path, input_col, output_col, key_col, api_key):
    """
    读取 CSV 文件，逐行处理内容，并使用 JSON 缓存。
    """
    try:
        df = pd.read_csv(file_path)

        if input_col not in df.columns:
            print(f"错误: CSV 文件中找不到列 '{input_col}'")
            return

        if key_col not in df.columns:
            print(f"警告: CSV 文件中找不到列 '{key_col}', 正在自动生成...")
            df[key_col] = [str(uuid.uuid4()) for _ in range(len(df))]

        cache = load_cache()
        processed_keys = set(cache.keys())
        
        # Prepare a list for batch processing
        texts_to_process = []
        keys_to_process = []
        
        print("开始处理 CSV 文件...")
        
        for index, row in df.iterrows():
            key = str(row[key_col])
            
            if key not in processed_keys:
                texts_to_process.append(row[input_col])
                keys_to_process.append(key)

                if len(texts_to_process) >= BATCH_SIZE or index == len(df) - 1:
                    print(f"正在处理一个包含 {len(texts_to_process)} 条记录的批次...")
                    results = call_api_batch(texts_to_process, api_key)
                    
                    for i in range(len(keys_to_process)):
                        key_to_update = keys_to_process[i]
                        result_from_batch = results[i]
                        cache[key_to_update] = result_from_batch
                        processed_keys.add(key_to_update)
                    
                    save_cache(cache)
                    
                    texts_to_process = []
                    keys_to_process = []
        
        print("所有数据处理完毕。正在将结果写入 CSV 文件...")
        
        for index, row in df.iterrows():
            key = str(row[key_col])
            if key in cache:
                df.at[index, output_col] = cache[key]
            else:
                df.at[index, output_col] = "Error: Not processed"
                
        output_file_path = file_path.replace(".csv", "_processed_cached.csv")
        df.to_csv(output_file_path, index=False, encoding='utf-8')
        print(f"处理完成！结果已保存到 '{output_file_path}'")

    except FileNotFoundError:
        print(f"错误: 文件 '{file_path}' 不存在。")
    except Exception as e:
        print(f"发生了一个错误: {e}")

# 运行主程序
if __name__ == "__main__":
    process_csv_with_cache(FILE_PATH, INPUT_COLUMN, OUTPUT_COLUMN, KEY_COLUMN, API_KEY)

In [7]:


# 从文件导入 JSON
with open("/Users/hemengya/Desktop/数字贸易规则新版/第五版/qwen/cache.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

# 转换为两列 DataFrame
df = pd.DataFrame(list(data.items()), columns=['key', 'value'])

# 如果 key 是数字字符串，可以尝试排序
try:
    df['key_num'] = df['key'].astype(int)
    df = df.sort_values('key_num').drop(columns='key_num')
except Exception:
    pass

# 输出到 Excel
df.to_excel("/Users/hemengya/Desktop/数字贸易规则新版/第五版/qwen/output.xlsx", index=False)
print("转换完成，已生成 output.xlsx")


转换完成，已生成 output.xlsx
